In [1]:
import numpy as np
import awkward0 as awkward

In [2]:
SEED = 0 # random seed corresponds to the member of the ensemble

In [3]:
import os
import tensorflow as tf

# uncomment and adjust for GPU calculations
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
# # dynamic memory growth
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
from os.path import join
from ROOT import TLorentzVector
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
import seaborn as sns
from tensorflow.keras.utils import *
import shutil
from dataset import *
import datetime

Welcome to JupyROOT 6.24/02


In [6]:
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [8]:
NO_OF_PARTICLES = 250 # how many particles in an event to use, default 250
PT_CUT = 1.0 # Lower cut on PT particles, default is 1.0
OUTPUT = f'model-{SEED}' # name of the output folder

In [9]:
if not os.path.exists(OUTPUT):
  # Create a new directory because it does not exist 
  os.makedirs(OUTPUT)

In [10]:
import logging
logging.basicConfig(filename=join(OUTPUT, 'training.log'),
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

In [11]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [12]:
shutil.copy('train-ensemble.ipynb', join(OUTPUT, 'train-ensemble.ipynb')) # copy notebook to the output dir

'attempt143-8/keras_train5-ENSEMBLE.ipynb'

## Load validation set. Adjust the data folder if needed.

In [14]:
val_dataset = Dataset(f'/data/higgsino_train/ensemble_val_{SEED}.awkd', {}, data_format='channel_last', simple_mode=False, pad_len=NO_OF_PARTICLES)

Keys in the datafile found:
['track_momentum', 'track_pt', 'track_phi', 'track_eta', 'track_charge', 'track_origin', 'photon_energy', 'photon_ET', 'photon_phi', 'photon_eta', 'photon_charge', 'photon_origin', 'hadron_energy', 'hadron_ET', 'hadron_phi', 'hadron_eta', 'hadron_charge', 'hadron_origin', 'event_pt', 'event_eta', 'event_phi', 'event_mass', 'label', 'train_val_test', 'n_parts', 'jet_PT', 'jet_Eta', 'jet_Phi', 'jet_Mass', 'jet_Flavor', 'con_jet1_phi_arr', 'con_jet2_phi_arr', 'con_jet3_phi_arr', 'con_jet1_eta_arr', 'con_jet2_eta_arr', 'con_jet3_eta_arr', 'con_jet1_pt_arr', 'con_jet2_pt_arr', 'con_jet3_pt_arr', 'con_jet1_type_arr', 'con_jet2_type_arr', 'con_jet3_type_arr', 'origin', 'part_pt_log', 'part_ptrel', 'part_logptrel', 'part_e_log', 'part_erel', 'part_logerel', 'squark_pt', 'part_raw_etarel', 'part_etarel', 'part_phirel', 'part_deltaR', 'event_MET', 'event_HT', 'event_MT2']
Loading ['part_etarel', 'part_phirel']
Loading ['part_etarel', 'part_phirel']
Loading ['part_pt_l

## Load training set. Adjust the data folder if needed.

In [15]:
train_dataset = Dataset(f'/eflow_data/higgsino_train/ensemble_train_{SEED}.awkd', {}, data_format='channel_last', simple_mode=False, pad_len=NO_OF_PARTICLES)

Keys in the datafile found:
['track_momentum', 'track_pt', 'track_phi', 'track_eta', 'track_charge', 'track_origin', 'photon_energy', 'photon_ET', 'photon_phi', 'photon_eta', 'photon_charge', 'photon_origin', 'hadron_energy', 'hadron_ET', 'hadron_phi', 'hadron_eta', 'hadron_charge', 'hadron_origin', 'event_pt', 'event_eta', 'event_phi', 'event_mass', 'label', 'train_val_test', 'n_parts', 'jet_PT', 'jet_Eta', 'jet_Phi', 'jet_Mass', 'jet_Flavor', 'con_jet1_phi_arr', 'con_jet2_phi_arr', 'con_jet3_phi_arr', 'con_jet1_eta_arr', 'con_jet2_eta_arr', 'con_jet3_eta_arr', 'con_jet1_pt_arr', 'con_jet2_pt_arr', 'con_jet3_pt_arr', 'con_jet1_type_arr', 'con_jet2_type_arr', 'con_jet3_type_arr', 'origin', 'part_pt_log', 'part_ptrel', 'part_logptrel', 'part_e_log', 'part_erel', 'part_logerel', 'squark_pt', 'part_raw_etarel', 'part_etarel', 'part_phirel', 'part_deltaR', 'event_MET', 'event_HT', 'event_MT2']
Loading ['part_etarel', 'part_phirel']
Loading ['part_etarel', 'part_phirel']
Loading ['part_pt_l

## Calculate means and standard deviations, and normalise the data.

In [16]:
scalers = train_dataset.normalize_all(scalers = None)

In [19]:
val_dataset.normalize_all(scalers=scalers)

{'scaler_phi': StandardScaler(),
 'scaler_eta': StandardScaler(),
 'scaler_met': StandardScaler(),
 'scaler_MT2': StandardScaler(),
 'scaler_ht': StandardScaler(),
 'scaler_m': StandardScaler(),
 'scaler_jet1PT': StandardScaler(),
 'scaler_jet1ETA': StandardScaler(),
 'scaler_jet1MASS': StandardScaler(),
 'scaler_jet1DPHI': StandardScaler(),
 'scaler_jet2PT': StandardScaler(),
 'scaler_jet2ETA': StandardScaler(),
 'scaler_jet2MASS': StandardScaler(),
 'scaler_jet2DPHI': StandardScaler(),
 'scaler_jet3PT': StandardScaler(),
 'scaler_jet3ETA': StandardScaler(),
 'scaler_jet3MASS': StandardScaler(),
 'scaler_jet3DPHI': StandardScaler(),
 'scaler_jet4PT': StandardScaler(),
 'scaler_jet4ETA': StandardScaler(),
 'scaler_jet4MASS': StandardScaler(),
 'scaler_jet4DPHI': StandardScaler()}

In [20]:
string = ''
for k,v in scalers.items():
    string += k + '\n'
    string += f'mean: {v.mean_[0]}'+ '\n'
    string += f'var: {v.var_[0]}' + '\n'
    string += f'scale: {v.scale_[0]}' + '\n'
print(string)
with open(join(OUTPUT, 'scaler.txt'), 'w') as f:
    f.write(string)

scaler_phi
mean: -0.0009872796078240246
var: 3.2911722820461997
scale: 1.814158835947448
scaler_eta
mean: 0.0009836877373430276
var: 0.6764734905577952
scale: 0.8224800852043745
scaler_met
mean: 1130.113519509007
var: 84220.0697576037
scale: 290.20694298655866
scaler_MT2
mean: 1007.596262032176
var: 82579.14149917677
scale: 287.36586696957727
scaler_ht
mean: 1546.1682843649069
var: 231655.84256466498
scale: 481.30639156847377
scaler_m
mean: 3052.7427181564426
var: 977431.4137292544
scale: 988.6513104878051
scaler_jet1PT
mean: 908.6745824742735
var: 107958.66982526249
scale: 328.5706466275746
scaler_jet1ETA
mean: 0.0005679062923058418
var: 0.5543616500942882
scale: 0.7445546656185079
scaler_jet1MASS
mean: 77.5006601535009
var: 2698.2103036196477
scale: 51.94430001087365
scaler_jet1DPHI
mean: -0.000466294407694618
var: 0.2287676683491307
scale: 0.4782966321741464
scaler_jet2PT
mean: 527.7618575955822
var: 43028.34287538092
scale: 207.43274301657615
scaler_jet2ETA
mean: 0.0011419683274109

## Apply PT cut

In [22]:
def apply_pt_cut(data, cut=PT_CUT):
    cond = data.X['mask'] < np.log(cut)
    cond2 = data.X['mask'] >= np.log(cut)
    print(cond.shape)
    print('below the cut: ', np.sum(cond))
    print('above the cut: ', np.sum(cond2))
    print('sum: ', np.sum(cond) + np.sum(cond2))
    data.X['mask'][cond] = 0.0
    ext_shape = list(cond.shape)
    ext_shape[-1] = 4
    new_cond = np.repeat(cond, 5, axis=2)
    print(new_cond.shape)
    data.X['features'][new_cond] = 0.0
  
    return data, np.sum(cond2, axis=1).flatten()

In [23]:
train_dataset, _ = apply_pt_cut(train_dataset, PT_CUT)
val_dataset, _ = apply_pt_cut(val_dataset, PT_CUT)

(817314, 250, 1)
below the cut:  181989564
above the cut:  22338936
sum:  204328500
(817314, 250, 5)
(350277, 250, 1)
below the cut:  78000615
above the cut:  9568635
sum:  87569250
(350277, 250, 5)


# Load the ParticleNet model

In [24]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [25]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = train_dataset.y.shape[1]
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
print(input_shapes)
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

{'points': (250, 2), 'features': (250, 5), 'mask': (250, 1), 'origin': (250, 1), 'track': (250, 3), 'photon': (250, 3), 'hadron': (250, 3), 'jet': (4, 5), 'squark': (3, 1), 'jet1_con': (80, 4), 'jet2_con': (80, 4), 'jet3_con': (80, 4)}


# Create a dense network for high-level variables

In [27]:
model2_inputs = keras.Input(shape=(5+4*4,))
xx = model2_inputs
xx = keras.layers.Dense(256, activation=None, kernel_regularizer=keras.regularizers.L2(1e-3))(xx)
xx = keras.layers.BatchNormalization()(xx)
xx = keras.layers.Activation(tf.nn.relu)(xx)
for ii in range(5):
    xx = keras.layers.Dense(128, activation=None, kernel_regularizer=keras.regularizers.L2(1e-3))(xx)
    xx = keras.layers.BatchNormalization()(xx)
    xx = keras.layers.Activation(tf.nn.relu)(xx)    
xx = keras.layers.Dropout(0.5)(xx)
xx = keras.layers.Dense(2, activation="softmax",)(xx)

model2 = keras.Model(model2_inputs, xx)

In [28]:
model2.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=5e-4), #lr_scheduler),
              metrics = [tf.keras.metrics.BinaryAccuracy()],
              weighted_metrics=[tf.keras.metrics.AUC()])


## Modify the GNN part

In [29]:
#Improved particle net
yy = model.layers[-4].output
yy = keras.layers.Dense(256, activation=None, kernel_regularizer=keras.regularizers.L2(1e-3))(yy)
yy = keras.layers.BatchNormalization()(yy)
yy = keras.layers.Activation(tf.nn.relu)(yy)
yy = keras.layers.Dropout(0.5)(yy)
yy = keras.layers.Dense(128, activation=None, kernel_regularizer=keras.regularizers.L2(1e-3))(yy)
yy = keras.layers.BatchNormalization()(yy)
yy = keras.layers.Activation(tf.nn.relu)(yy)

model3 = keras.Model(model.input, yy)

# Merge two models

In [30]:
merged_layer = keras.layers.Concatenate()([model2.layers[-3].output, model3.layers[-1].output])
z = merged_layer
z = keras.layers.Dense(128, activation=None, kernel_regularizer=keras.regularizers.L2(1e-3))(z)
z = keras.layers.BatchNormalization()(z)
z = keras.layers.Activation(tf.nn.relu)(z)
z = keras.layers.Dropout(0.5)(z)
z = keras.layers.Dense(2, activation="softmax",)(z)
merged_model = keras.Model([model2.input, model3.input], z)

## Set the batch size

In [32]:
batch_size = 1024

# Warmup cosine decay

In [33]:
# based on https://stackabuse.com/learning-rate-warmup-with-cosine-decay-in-keras-and-tensorflow/
from tensorflow.keras import backend as K

def lr_warmup_cosine_decay(global_step,
                           warmup_steps,
                           hold = 0,
                           total_steps=500,
                           start_lr=0.0,
                           target_lr=1e-3,
                           final_lr=1e-5):
    # Cosine decay
    # There is no tf.pi so we wrap np.pi as a TF constant
    learning_rate = final_lr+ 0.5 * target_lr * (1.0 + tf.cos(tf.constant(np.pi) * tf.convert_to_tensor(global_step - warmup_steps - hold, dtype=np.float32) / tf.convert_to_tensor(total_steps - warmup_steps - hold, dtype=np.float32)))

    # Target LR * progress of warmup (=1 at the final warmup step)
    warmup_lr = target_lr * (global_step / warmup_steps)

    # Choose between `warmup_lr`, `target_lr` and `learning_rate` based on whether `global_step < warmup_steps` and we're still holding.
    # i.e. warm up if we're still warming up and use cosine decayed lr otherwise
    if hold > 0:
        learning_rate = tf.where(global_step > warmup_steps + hold,
                                 learning_rate, target_lr)
    
    learning_rate = tf.where(global_step < warmup_steps, warmup_lr, learning_rate)

    return learning_rate



class WarmUpCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, start_lr, target_lr, warmup_steps, total_steps, hold, final_lr):
        super().__init__()
        self.start_lr = start_lr
        self.target_lr = target_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.hold = hold
        self.final_lr = final_lr
    
    def get_config(self):
        config = {
          'start_lr': self.start_lr,
          'target_lr': self.target_lr,
          'warmup_steps': self.warmup_steps,
          'total_steps': self.total_steps,
          'hold': self.hold,
          'final_lr':self.final_lr,
        }
        return config

    def __call__(self, step):
        lr = lr_warmup_cosine_decay(global_step=tf.cast(step, np.float32),
                                    total_steps=self.total_steps,
                                    warmup_steps=self.warmup_steps,
                                    start_lr=self.start_lr,
                                    target_lr=self.target_lr,
                                    hold=self.hold,
                                    final_lr=self.final_lr)

        return tf.where(
            step > self.total_steps, self.final_lr, lr, name="learning_rate"
        )


In [34]:
lr_schedule = WarmUpCosineDecay(start_lr=0, target_lr=5e-4, warmup_steps=10*800, total_steps=310*800, hold=0, final_lr=1e-5)

## Compile

In [35]:
merged_model.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=5e-4),
                     metrics = [tf.keras.metrics.BinaryAccuracy()],
                     weighted_metrics=[tf.keras.metrics.AUC()])
merged_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 250, 1)]     0           []                               
                                                                                                  
 tf.math.not_equal (TFOpLambda)  (None, 250, 1)      0           ['mask[0][0]']                   
                                                                                                  
 tf.cast (TFOpLambda)           (None, 250, 1)       0           ['tf.math.not_equal[0][0]']      
                                                                                                  
 tf.math.equal (TFOpLambda)     (None, 250, 1)       0           ['tf.cast[0][0]']                
                                                                                            

In [36]:
# Prepare model checkpoint directory.
import os
save_dir = join(OUTPUT, 'model_checkpoints')
model_name = 'model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

progress_bar = keras.callbacks.ProgbarLogger()


In [37]:
# define early stopping
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=50,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [38]:
callbacks = [checkpoint, progress_bar, earlystop,]

# Training

In [39]:
bkg_n = np.sum(train_dataset.y[:,1] == 1.0)
sig_n = np.sum(train_dataset.y[:,0] == 1.0)
total = bkg_n + sig_n
print(bkg_n, sig_n, total)
weight_for_bkg = (1 / bkg_n) * (total / 2.0)
weight_for_sig = (1 / sig_n) * (total / 2.0)
class_weights = {0: weight_for_sig, 1: weight_for_bkg}
print(class_weights) # dataset is very well balanced but we use weights anyway


411001 406313 817314
{0: 1.0057689515225947, 1: 0.9942968508592436}


In [40]:
train_dataset.shuffle()

## Handle high-level data

In [42]:
curr_dataset = train_dataset
dnn_vars = [curr_dataset['event_met'], curr_dataset['event_ht'], curr_dataset['event_eta'], curr_dataset['event_m'], curr_dataset['event_MT2'], ]
for ii in range(0, 4):
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 0])
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 1])
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 3])
    dnn_vars.append(curr_dataset['Dphi'][:, ii])
model2_train_X = list(zip(*dnn_vars))
model2_train_X = np.array(model2_train_X)
model2_train_X.shape


(817314, 21)

In [43]:
curr_dataset = val_dataset
dnn_vars = [curr_dataset['event_met'], curr_dataset['event_ht'], curr_dataset['event_eta'], curr_dataset['event_m'], curr_dataset['event_MT2'], ]
for ii in range(4):
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 0])
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 1])
    dnn_vars.append(curr_dataset.X['jet'][:, ii, 3])
    dnn_vars.append(curr_dataset['Dphi'][:, ii])

model2_val_X = list(zip(*dnn_vars))
model2_val_X = np.array(model2_val_X)
model2_val_X.shape

(350277, 21)

In [44]:
train_dataset.X['features'].shape

(817314, 250, 5)

In [ ]:
# Train the network

In [ ]:
history = merged_model.fit([model2_train_X, train_dataset.X['points'], train_dataset.X['features'], train_dataset.X['mask']], train_dataset.y,
          batch_size=batch_size,
          epochs=3000,
          validation_data=([model2_val_X, val_dataset.X['points'],  val_dataset.X['features'],  val_dataset.X['mask'],], val_dataset.y),
          class_weight=class_weights,
          shuffle=True,
          verbose=2,                 
          callbacks=callbacks)

Epoch 1/3000

Epoch 00001: val_loss improved from inf to 1.74355, saving model to attempt143-8/model_checkpoints/model.001.h5
780/780 - 385s - loss: 1.9289 - binary_accuracy: 0.5552 - auc_1: 0.5835 - val_loss: 1.7436 - val_binary_accuracy: 0.6199 - val_auc_1: 0.6745 - 385s/epoch - 493ms/sample
Epoch 2/3000


In [ ]:
merged_model.save(OUTPUT) # save it to disk

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss function of the model')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('{}/loss.pdf'.format(OUTPUT))
plt.show()

In [ ]:
# accuracy
fig = plt.figure()
plt.plot(history.history['binary_accuracy'], label='train')
plt.plot(history.history['val_binary_accuracy'], label='val')
plt.title('Accuracy of the model')
plt.xlabel('epoch')
plt.ylabel('binary accuracy')
plt.legend()
plt.hlines(xmin=1, xmax=len(history.history['binary_accuracy']), y=0.8, color='green', linestyle='--')
plt.hlines(xmin=1, xmax=len(history.history['binary_accuracy']), y=0.9, color='green', linestyle=':')
plt.vlines(ymin=np.min(history.history['val_binary_accuracy']), ymax=0.9, x = len(history.history['val_binary_accuracy'])-51.5, color='red',linestyle=':')
plt.savefig('{}/accuracy.pdf'.format(OUTPUT))
# plt.show()
# plt.savefig('{}/score.pdf'.format(OUTPUT))
# history.history['accuracy']

In [ ]:
predictions = merged_model.predict([model2_val_X, val_dataset.X['points'], val_dataset.X['features'], val_dataset.X['mask']])

In [ ]:
predictions[:10]

In [ ]:
y_pred = predictions[:, 0]
y_true = np.array([1 if v[0]>v[1] else 0 for v in val_dataset.y], dtype=np.single)

In [ ]:
plt.title('Predicted score on val set')
plt.hist(y_pred[y_true==1], bins=30, histtype='stepfilled', alpha=0.3, label='w300_sq2200')
plt.hist(y_pred[y_true==0], bins=30, histtype='stepfilled', alpha=0.3, label='SM')
plt.yscale('log')
plt.legend()
plt.xlabel('score')
plt.ylabel("MC events")
# plt.show()
plt.savefig('{}/score.pdf'.format(OUTPUT))

In [ ]:
plt.title('Predicted score on val set (scaled with xs)')
sig_score_weights = np.full(y_pred[y_true==1].shape, 3000*sig_xs/y_pred[y_true==1].shape[0])
plt.hist(y_pred[y_true==1], bins=30, histtype='stepfilled', weights=sig_score_weights, alpha=0.3, label='w300_sq2200')
bkg_score_weights = np.full(y_pred[y_true==0].shape, 3000*bkg_xs/y_pred[y_true==0].shape[0])
plt.hist(y_pred[y_true==0], bins=30, histtype='stepfilled', weights=bkg_score_weights, alpha=0.3, label='SM')
plt.yscale('log')
plt.legend()
plt.xlabel('score')
plt.ylabel("Events for HL-LHC")
# plt.show()
plt.savefig('{}/score_norm.pdf'.format(OUTPUT))

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(join(OUTPUT, hist_json_file), mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(join(OUTPUT, hist_csv_file), mode='w') as f:
    hist_df.to_csv(f)


In [ ]:
# accuracy
fig = plt.figure()
plt.plot(history.history['binary_accuracy'], label='train acc', color='blue', linestyle='-')
plt.plot(history.history['val_binary_accuracy'], label='val acc', color='blue', linestyle='--')
plt.plot(history.history['auc_1'], label='train AUC', color='red', linestyle='-')
plt.plot(history.history['val_auc_1'], label='val AUC', color='red', linestyle='--')

plt.title('Accuracy of the model')
plt.xlabel('epoch')
plt.ylabel('binary accuracy')
plt.legend()
plt.ylim(0.5, 1.05)
plt.vlines(ymin=0.5, ymax=1.05, x = len(history.history['val_binary_accuracy'])-51.5, color='green',linestyle=':')

plt.grid()
plt.savefig('{}/acc_auc.pdf'.format(OUTPUT))